# 卷积层与批归一化层的融合推导

当我们将卷积层（Conv）与批归一化层（BatchNorm, BN）融合时，BN 的参数会被合并到卷积层的权重和偏置中。以下是详细的推导过程。

## 1. 卷积层输出

设卷积层的权重和偏置为：

- 权重：\( W \)
- 偏置：\( b \)

输入为 \( x \)，卷积层的输出为：

$$
y = W \cdot x + b
$$

## 2. 批归一化层

批归一化层的参数为：

- 伽马：\( \gamma \)
- 贝塔：\( \beta \)
- 均值：\( \mu \)
- 方差：\( \sigma^2 \)

在批归一化中，输出为：

$$
\text{BN}(y) = \gamma \left( \frac{y - \mu}{\sqrt{\sigma^2 + \epsilon}} \right) + \beta
$$

其中，\( \epsilon \) 是一个小的常数，防止除零错误。

## 3. 将卷积输出代入批归一化

将卷积输出 \( y \) 代入批归一化公式：

$$
\text{BN}(y) = \gamma \left( \frac{(W \cdot x + b) - \mu}{\sqrt{\sigma^2 + \epsilon}} \right) + \beta
$$

## 4. 展开和整理

展开公式后，可以得到：

$$
\text{BN}(y) = \frac{\gamma}{\sqrt{\sigma^2 + \epsilon}} (W \cdot x + b) - \frac{\gamma \mu}{\sqrt{\sigma^2 + \epsilon}} + \beta
$$

## 5. 定义新的卷积参数

我们可以定义新的卷积层参数：

- 新的权重：

$$
W' = \frac{\gamma}{\sqrt{\sigma^2 + \epsilon}} W
$$

- 新的偏置：

$$
b' = \frac{\gamma}{\sqrt{\sigma^2 + \epsilon}} b - \frac{\gamma \mu}{\sqrt{\sigma^2 + \epsilon}} + \beta
$$

## 6. 完整的输出表达式

融合后的输出可以表示为：

$$
\text{FusedOutput}(x) = W' \cdot x + b'
$$

## 结论

通过上述推导，我们可以看到 `BatchNorm` 的参数已经成功融合到了卷积层的权重和偏置中。这种融合可以简化模型并提高推理性能。

In [60]:
import torch
import torch.nn as nn

# 创建卷积和批归一化层
conv = nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1)
bn = nn.BatchNorm2d(1)

# 随机输入
x = torch.randn(1, 1, 5, 5)

# 原始的卷积和批归一化输出
conv_output = conv(x)
bn.train()  # 设置为训练模式，计算当前批次的均值和方差
bn_output_train = bn(conv_output)

manual_batch_mean = torch.mean(conv_output, dim=[0, 2, 3])
# 注意训练模式下，当前batch的方差是无偏的
manual_batch_var = torch.var(conv_output, dim=[0, 2, 3], unbiased=True)
print(f"手动计算批次均值: {manual_batch_mean}")
print(f"手动计算批次方差: {manual_batch_var}")

# 假设初始的运行均值和方差
init_running_mean = torch.zeros(1)  # 初始为0
init_running_var = torch.ones(1)  # 初始为1

# 设置 alpha
alpha = 0.9  # 通常使用较小的值

# 更新运行均值和方差
manual_running_mean = alpha * init_running_mean + (1 - alpha) * manual_batch_mean
manual_running_var = alpha * init_running_var + (1 - alpha) * manual_batch_var
print(f"手动计算的运行均值: {manual_running_mean}")
print(f"手动计算的运行方差: {manual_running_var}")

# 切换到评估模式以使用运行均值和方差
bn.eval()
bn_output_eval = bn(conv_output)

# 计算运行均值和方差
running_mean = bn.running_mean
# 测试时，当前
running_var = bn.running_var
print(f"评估模式下，计算的运行均值: {running_mean}")
print(f"评估模式下，计算的运行方差: {running_var}")


# 融合后的参数计算
gamma = bn.weight
beta = bn.bias
mean = bn.running_mean
var = bn.running_var
epsilon = bn.eps

# 计算融合后的参数
weight_fused = (gamma / (var + epsilon).sqrt()) * conv.weight
bias_fused = (gamma / (var + epsilon).sqrt()) * conv.bias - (gamma * mean / (var + epsilon).sqrt()) + beta

# 计算融合后的输出
fused_output = torch.nn.functional.conv2d(x, weight_fused, bias_fused, stride=1, padding=1)

# 验证
is_equal = torch.allclose(bn_output_eval, fused_output, atol=1e-6)
print(f"Outputs are equal: {is_equal}")  # 应为 True

手动计算批次均值: tensor([-0.0680], grad_fn=<MeanBackward1>)
手动计算批次方差: tensor([0.2569], grad_fn=<VarBackward0>)
手动计算的运行均值: tensor([-0.0068], grad_fn=<AddBackward0>)
手动计算的运行方差: tensor([0.9257], grad_fn=<AddBackward0>)
评估模式下，计算的运行均值: tensor([-0.0068])
评估模式下，计算的运行方差: tensor([0.9257])
Outputs are equal: True


In [61]:
import torch
import torch.nn as nn

# 创建卷积和批归一化层
conv = nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1)
bn = nn.BatchNorm2d(1)

# 假设初始的运行均值和方差
init_running_mean = torch.zeros(1)  # 初始为0
init_running_var = torch.ones(1)  # 初始为1

# 设置 alpha
alpha = 0.9  # 通常使用较小的值

# 用于记录每个批次的均值和方差
batch_means = []
batch_vars = []

# 输入数据：10个批次
num_batches = 10
batch_size = 16
input_shape = (batch_size, 1, 5, 5)

# 初始化运行均值和方差
running_mean = init_running_mean.clone()
running_var = init_running_var.clone()

for i in range(num_batches):
    # 生成随机输入
    x = torch.randn(input_shape)
    
    # 训练模式下处理当前批次
    conv_output = conv(x)
    bn.train()
    bn_output_train = bn(conv_output)

    # 计算当前批次的均值和方差
    batch_mean = torch.mean(conv_output, dim=[0, 2, 3])
    # 注意这里当前batch的方差是基于无偏估计（若使用有偏估计，则手动与运行方差结果不一致）
    # 但是训练模式下，running_var 是通过当前批次的有偏方差（使用 N 作为分母）进行更新的，这里矛盾？
    batch_var = torch.var(conv_output, dim=[0, 2, 3], unbiased=True)

    # 记录均值和方差
    batch_means.append(batch_mean)
    batch_vars.append(batch_var)

    # 更新运行均值和方差
    running_mean = alpha * running_mean + (1 - alpha) * batch_mean
    running_var = alpha * running_var + (1 - alpha) * batch_var

# 切换到评估模式
bn.eval()
# 最后一个批次的输出用于评估
bn_output_eval = bn(conv_output)

# 获取评估模式下的运行均值和方差
eval_running_mean = bn.running_mean
eval_running_var = bn.running_var

# 手动计算滑动平均的运行均值和方差
manual_running_mean = init_running_mean.clone()
manual_running_var = init_running_var.clone()

for mean, var in zip(batch_means, batch_vars):
    manual_running_mean = alpha * manual_running_mean + (1 - alpha) * mean
    manual_running_var = alpha * manual_running_var + (1 - alpha) * var

# 输出结果
print(f"评估模式下，计算的运行均值: {eval_running_mean.item()}")
print(f"评估模式下，计算的运行方差: {eval_running_var.item()}")

print(f"手动计算的滑动平均运行均值: {manual_running_mean.item()}")
print(f"手动计算的滑动平均运行方差: {manual_running_var.item()}")

# 比较结果
mean_equal = torch.allclose(eval_running_mean, manual_running_mean)
var_equal = torch.allclose(eval_running_var, manual_running_var)

print(f"运行均值一致性: {mean_equal}")
print(f"运行方差一致性: {var_equal}")

评估模式下，计算的运行均值: 0.13081976771354675
评估模式下，计算的运行方差: 0.4471468925476074
手动计算的滑动平均运行均值: 0.13081976771354675
手动计算的滑动平均运行方差: 0.4471468925476074
运行均值一致性: True
运行方差一致性: True
